## 생산계획 팀프로젝트 - I조

2018170817 이재혁\
2018170819 송찬재\
2018170820 정종헌\
2018170839 이건희


MPS와 MRP는 ERP 시스템의 엔진에 해당한다고 할 수 있다.
제품에 대한 12개월 Forecast, BOM, 재고 정보 등이 주어졌을 때 다음을 조건을 만족하는 프로그램을 작성하라.

1) ATP를 포함한 MPS 계획과, 그 MPS를 기반으로 모든 부품에 대한 MRP 계획을 전개\
2) 1st & 2nd month 판매, 생산 실적(Actual)이 주어졌을 때 MPS와 MRP 계획을 update

- MTO 경영환경 가정
- 각 제품단계의 MPS에서는 Batch 또는 Chasing 생산 전략을 선택하고, 부품의 MRP 계획에서도 Batch 또는 Lot for lot 생산 전략을 선택한다.
- 제품 A, B, C의 Forecast, Order을 Input 값으로 받으면 MRP가 전개되도록 프로그래밍한다.

## 생산 전략: Chase Demand / L4L

## Import 및 Queue 이용을 위한 정의
- Queue.clear 함수를 추가해 update 기능 구현에 활용하였습니다.

In [1]:
import os.path
import csv
import re
import textwrap

In [2]:
class Queue:
    def __init__(self):
        self.front = None
        self.rear = None
        self.num_items = 0

    def __eq__(self, other):
        return ((type(other) == Queue)
            and self.capacity == other.capacity
            and self.front == other.front
             )
   
    def __repr__(self, other):
        return ("Queue({!r}, {!r})".format(self.capacity, self.front))

    def is_empty(self):
        return self.num_items == 0

    def add(self, item):
        if self.is_empty():
            self.front = item
        else:
            self.rear.next = item
        self.rear = item
        self.num_items += 1

    def get(self):
        item = self.front
        if self.num_items > 1:
            self.front = self.front.next
        else:
            self.front = None
            self.rear = None
        self.num_items -= 1
        return item
    
    def clear(self):
        self.front=None
        self.rear=None
        self.num_items=0

## part - 클래스 생성
각 부품 별 정보를 part 클래스의 instance로 저장 - 원하는 정보를 추가적으로 저장하도록 수정하였습니다.

- 다음 정보는 int 값으로 저장\
on-hand, safety stock, lead time


- 다음 정보는 Dictionary 로 저장\
BOM, MPS(gross requirement), Forecast, Order, MRP(Planned order release), PAB, Planned order receipt, ATP


- 추가적으로 list로 저장\
MRP, PAB

In [3]:
class part:
    def __init__(self, name, oh, alloc, ss, lt, ls):
        self.name = name
        self.oh = oh       # on-hand --> will be pab_dict{0:oh}
        self.alloc = alloc
        self.ss = ss       # safety stock
        self.lt = lt       # lead time
        self.ls = ls       # lead time sensitivity
        self.sr = {}       # scheduled receipt {period : sr}
        self.bom = {}      # bom-children
        self.mps = {}      # mps {period : gross requirement}
        self.fc = {}       # forecast {period : forecast}
        self.od = {}       # order {period : order}
        self.mrp = []      # planned order release
        self.mrp_dict = {} # planned order release {period : mrp} 
        self.avl = []      # projected available balance
        self.pab_dict = {} # PAB {period : pab} - 0 : oh
        self.pr_dict = {}  # planned order receipt {period : pr}
        self.atp = {}      # ATP {period : atp} --> only for MPS / '-' means None
        
        self.prnt_bu = 0
        self.prnt = 0
        self.queued = False
        self.next = next
 
    def __eq__(self, other):
        return ((type(other) == part)
                 and self.name == other.name
                 and self.oh == other.oh
                 and self.alloc == other.alloc
                 and self.ss == other.ss
                 and self.lt == other.lt
                 and self.ls == other.ls
                 and self.sr == other.sr
                 and self.bom == other.bom
                 and self.mps == other.mps
                 and self.fc == other.fc
                 and self.od == other.od
                 and self.mrp == other.mrp
                 and self.atp == other.atp     
                 and self.avl == other.avl
                 and self.prnt == other.prnt
                 and self.queued == other.queued
                 and self.next == other.next
                )

    def __hash__(self):
        return hash((self.name))

    def __repr__(self):
        return ("part({!r}, {!r}, {!r}, {!r}, {!r}, {!r}, {!r}, {!r}, {!r}, {!r}, {!r},{!r}, {!r})".format(self.name, self.oh, self.alloc, self.ss, self.lt, self.ls, self.sr, self.bom, self.mps, self.mrp, self.avl, self.prnt, self.queued))


## Initial Input을 받기 위한 함수
- imf, bom, mps 를 csv 파일로 받습니다.
- 폴더 내에 imf.csv, mps.csv, bom.csv 로 저장합니다.
- 폴더 명을 input으로 받습니다
- 일부 수정하였습니다.

In [4]:
def input_message(doc):
    fle = input('Please enter the name of the folder of ' + doc + ' you would like to use (csv files only): ')
    return fle


def imf_input():
    doc = input_message('IMF')
    PATH = doc
    if doc == 'data':
        doc = os.path.join(PATH, 'imf.csv')
    with open(doc) as file_:
        csv_ = csv.reader(file_, delimiter=',')
        skip = next(csv_)
        for line in csv_:
            line = [e.strip() for e in line]
            prt = part(str(line[0]), int(line[1]), int(line[2]), int(line[3]), int(line[4]), int(line[5]))
            try:
                sr = re.split('[: ]', line[6])
                sr = [int(e) for e in sr]
                prt.sr = dict(zip(sr[0::2], sr[1::2]))
            except ValueError:
                pass
            all_parts.append(prt)
            part_names[prt.name] = prt

            
def mps_input():
    global num_periods
    doc = input_message('MPS')
    PATH = doc
    if doc == 'data':
        doc = os.path.join(PATH,'mps.csv')
    with open(doc) as file_:
        csv_ = csv.reader(file_, delimiter=',')
        header = next(csv_)
        header = [e.strip() for e in header]
        periods = [int(e) for e in header[1::]]
        num_periods = max(periods)
        mn = min(periods)
        if mn <= 0:
            neg = 1 - mn
        for _ in range(3):
            header = next(csv_)
            header = [e.strip() for e in header]
            prt = part_names[header[0]]
         
            # forecast와 order info를 사용하기 위한 추가 코드
            prt.fc = dict(zip(periods, [int(e) for e in header[1:]]))
            # print(prt.fc)
            header = next(csv_)
            prt.od = dict(zip(periods, [int(e) for e in header[1:]]))
            # print(prt.od)         
            for with_fc, with_od in zip(prt.fc.items(), prt.od.items()):
                period, fc = with_fc
                period, od = with_od
                #prt.mps[period] = max(fc, od)

            
def bom_input():
    doc = input_message('BOM')
    PATH = doc
    if doc == 'data':
        doc = os.path.join(PATH, 'bom.csv')
    with open(doc) as file_:
        csv_ = csv.reader(file_, delimiter=',')
        skip = next(csv_)
        for line in csv_:
            line = [e.strip() for e in line]
            if len(line) == 2:
                chldrn = re.split('[: ]', line[1])
                ch_names = [str(e) for e in chldrn[0::2]]
                ch_qt = [int(e) for e in chldrn[1::2]]
                parent = part_names[line[0]]
                for i, name in enumerate(ch_names):
                    chld = part_names[name]
                    chld.prnt += 1
                    parent.bom[chld] = ch_qt[i]

## BOM 그래프를 그리는 함수

In [5]:
def draw_bom(part, level):
    for child, qt in part.bom.items():
        print(' '+level*'|   '+'|- '+child.name.upper()+' ('+str(qt)+')')
        draw_bom(child, level +1)

## Solver 함수
Reference:https://github.com/twoldstad/mrpSolver
- IMF, MPS, BOM data를 바탕으로 MRP를 계산하는 함수

In [7]:
def solver():
    global all_parts
    global top_level
    global part_names
    global current_level
    global num_periods
    global neg
    
    for part in all_parts:
        if part.prnt == 0:
            current_level.add(part)
    
    while not current_level.is_empty():        
        current_part = current_level.get()
        if current_part.prnt == 0:            
            for child in current_part.bom:
                child.prnt -= 1
                if not child.queued:
                    current_level.add(child)
                    child.queued = True
                    
            # run calculations on current part
            need_lst = [0] * (num_periods + neg)
            release_lst = need_lst[::]
            avail_lst = need_lst[::]
            avail = 0
            first_add = False
            
            # go through demand for part and add to list that accounts for negative periods
            for period, qt in current_part.mps.items():
                 need_lst[int(period) + neg - 1] = qt

            # go through list of demand for part while maintaining correct period references
            for per, qt in enumerate(need_lst, start=(1-neg)):
                i_per = per + neg - 1
                avail += current_part.sr.get(per, 0)
                if per > 0 and first_add is False:
                    avail += current_part.oh
                    first_add = True
                if qt > 0:
                    leftover = avail - qt
                    if leftover < current_part.ss:
                        when = per - current_part.lt
                        i_when = when + neg - 1
                        if 1 - when > neg:
                            new_neg = 1 - when
                            neg_dif = new_neg - neg
                            release_lst = [0]*neg_dif + release_lst
                            need_lst = [0]*neg_dif + need_lst
                            avail_lst = [0]*neg_dif + avail_lst
                            i_per += neg_dif
                            i_when += neg_dif
                            neg = new_neg
                        if per > 0:
                            short = current_part.ss - leftover
                            # buy = (short // current_part.ls + (short % current_part.ls > 0)) * current_part.ls
                            buy = short
                        else:
                            buy = qt
                        avail += buy
                        release_lst[i_when] += buy
                        for child, amount in current_part.bom.items():
                            child.mps[when] = amount*buy + child.mps.get(when, 0)
                    avail -= qt
                avail_lst[i_per] = avail
            current_part.mrp = release_lst
            current_part.avl = avail_lst 
        else:
            current_level.add(current_part)

## 추가 Solver 함수 - 직접 작성

- Solver에서 계산된 Data를 {Period: value}의 딕셔너리로 part 클래스 내부에 저장
- Planned receipt / PAB / ATP 를 추가적으로 계산

In [8]:
def add_solver():
    
    global num_periods
    
    total_periods = num_periods + neg
    for part in all_parts:
        if len(part.mrp) < total_periods:
            part.mrp = [0] * (total_periods - len(part.mrp)) + part.mrp
        if len(part.avl) < total_periods:
            part.avl = [0] * (total_periods - len(part.avl)) + part.avl
            
    for part in all_parts:
        
        # MRP - dict 
        # Planned order release
        part.mrp_dict = dict(zip(list(i for i in range(1, num_periods+1)),
                              part.mrp))
        
        # Planned receipt - dict
        part.pr_dict = dict(zip(list(i for i in range(1, num_periods+1)),
                             list(0 for _ in range(1, num_periods+1))))
        for x in range(1, num_periods - part.lt + 1):
            part.pr_dict.update({x+part.lt : part.mrp_dict[x]})
        
        # PAB - dict (0:OH  추가)
        part.pab_dict = dict(zip(list(i for i in range(1, num_periods+1)),
                                part.avl))
        part.pab_dict.update({0:part.oh})
        
        
        # ATP 계산 - finished prodcut 인 A, B, C에 대해서만
        if part.name in ['a', 'b', 'c']:
            for x in part.mrp_dict:
                if part.pr_dict[x] != 0:
                    part.atp.update({x:0})
            initial_atp = part.pab_dict[0] + part.pr_dict[1] - part.od[1]
            if 1 in part.sr:
                initial_atp += part.sr[1]
            part.atp.update({1:initial_atp})
            
            current = 1
            for t in range(2, num_periods+1):
                if t in part.atp:
                    current = t
                    part.atp[current] = part.pr_dict[current] - part.od[t]
                    if t in part.sr:
                        part.atp[current] += part.sr[t]
                else:
                    part.atp[current] -= part.od[t]
                    

## part 별 정보 초기화
- Update 구현에 기존의 함수들을 활용하기 위해 작성
- all_parts 내 part들의 data를 계산을 위해 세팅
- Forecast, Order 바탕으로 MPS 계산 > Update 시 변경된 값으로 다시 계산해야 하므로
- 남기는 정보\
oh, alloc, ss, lt, ls, sr, bom, fc, od
- update 구현에서 Queue 재활용을 위한 clear

In [9]:
def set_parts():
    global all_parts
    global current_level
    global neg
    global neg_bu
    
    neg = neg_bu
    
    current_level.clear()
    
    for part in all_parts:
        part.mrp = []
        part.mrp_dict={}
        part.avl = []
        part.pab_dict = {}
        part.pr_dict = {}
        part.atp = {}
        part.mps = {}
        part.queued = False
        part.prnt = part.prnt_bu
        
        for t in part.fc:
            if t in part.od:
                part.mps[t] = max(part.fc[t], part.od[t])

## Chart 출력을 위한 함수
- Part 클래스에 저장된 정보를 불러와 출력
- ATP 계산 결과 음수일 때, 0으로 출력
- ATP 가 존재하지 않을 때(생산이 없을 때) -으로 출력
- MPS와 MRP의 레코드 간 차이 존재

In [10]:
def record_print():
    global num_periods

    cols = list(range(-neg, num_periods+1))
    top_width = 24 + 6*len(cols)

    for part in all_parts:
        if part.name in ['a', 'b', 'c']:
            part.fc_list = [0] * num_periods  # forecast list 전환
            for key, value in part.fc.items():
                part.fc_list[key - 1] = value

            part.od_list = [0] * num_periods  # order 전환
            for key, value in part.od.items():
                part.od_list[key - 1] = value            
            
            part.atp_list = ['-'] * num_periods # atp 전환
            for key, value in part.atp.items():
                part.atp_list[key - 1] = value
                if part.atp_list[key - 1] < 0: # 음수 방지
                    part.atp_list[key - 1] = 0

        part.mps_list = [0] * num_periods # gross requirement list 전환
        for key, value in part.mps.items():
            part.mps_list[key - 1] = value

        part.sr_list = [0] * num_periods # scheduled receipt list 전환
        for key, value in part.sr.items():
            part.sr_list[key - 1] = value
        
        part.pr_list = [0] * num_periods # planned receipt 전환
        for key, value in part.pr_dict.items():
            part.pr_list[key - 1] = value
        
        part.pab_list = [0 for _ in range(num_periods + 1)]
        for key, value in part.pab_dict.items():
            part.pab_list[key] = value
        
        part.net_list = [0] * num_periods # net requirement를 추가함
        for i in range(12):
            if i >= 1:
                part.net_list[i] = part.mps_list[i] - part.sr_list[i] - part.avl[i-1] + part.ss
                if part.net_list[i] < 0: # 음수 방지
                    part.net_list[i] = 0

        if part.name in ['a', 'b', 'c']:
            print('{:30s}'.format('MPS'))
        top_row = '{:24s}'.format('Product ' + part.name.upper() +'.') + ''.join('{:^6d}'.format(e) for e in cols) # 제목
        print(top_row)
        print('-' * top_width)

        # 출력
        if part.name in ['a', 'b', 'c']: # MPS
            print('{:30s}'.format('Forecasts') + ''.join('{:^6d}'.format(e) for e in part.fc_list))
            print('{:30s}'.format('Orders') + ''.join('{:^6d}'.format(e) for e in part.od_list))
        else: # MRP
            print('{:30s}'.format('Gross requirements')+''.join('{:^6d}'.format(e) for e in part.mps_list))

        print('{:30s}'.format('Scheduled receipts')+''.join('{:^6d}'.format(e) for e in part.sr_list))
        if part.name in ['a', 'b', 'c']:
            print('{:24s}'.format('PAB')+''.join('{:^6d}'.format(e) for e in part.pab_list))
        else:
            print('{:24s}'.format('PIB')+''.join('{:^6d}'.format(e) for e in part.pab_list))
    
        if part.name in ['a', 'b', 'c']:  
            print('{:30s}'.format('ATP')+''.join('{:^6}'.format(e) for e in part.atp_list))  
        
        print('{:30s}'.format('Net requirements')+''.join('{:^6d}'.format(e) for e in part.net_list))
        print('{:30s}'.format('Planned receipts')+''.join('{:^6d}'.format(e) for e in part.pr_list))
        print('{:30s}'.format('Planned order release')+''.join('{:^6d}'.format(e) for e in part.mrp))
        print('-' * top_width)
        print(f'lead time={part.lt}, safety stock={part.ss}')
        print('')

## Input Your Initial Data directory
- csv 파일이 존재하는 폴더명 입력
- 기본 세팅은 data 폴더 내에 csv 파일 존재
- 따라서 data를 3번 입력하면 됨

In [11]:
all_parts = []
top_level = []
part_names = {}
current_level = Queue()
num_periods = 0
neg = 0
notes = []

imf_input()
mps_input()
bom_input()

neg_bu = neg

for part in all_parts:
    part.prnt_bu = part.prnt
    if part.prnt == 0:
        top_level.append(part)

Please enter the name of the folder of IMF you would like to use (csv files only): data
Please enter the name of the folder of MPS you would like to use (csv files only): data
Please enter the name of the folder of BOM you would like to use (csv files only): data


## 주어진 Initial Input Data를 바탕으로 계산 및 Print

In [12]:
set_parts()
solver()
add_solver()

## BOM 출력

In [13]:
for part in top_level:
    print(part.name.upper())
    draw_bom(part, 0)
    print()

A
 |- D (1)
 |   |- E (2)
 |   |- F (3)
 |   |- G (1)

B
 |- E (2)
 |- H (1)
 |   |- F (2)
 |   |- I (3)

C
 |- G (2)
 |- J (1)
 |- H (2)
 |   |- F (2)
 |   |- I (3)



## MPS 및 MRP 출력

In [14]:
record_print()

MPS                           
Product A.                0     1     2     3     4     5     6     7     8     9     10    11    12  
------------------------------------------------------------------------------------------------------
Forecasts                      100   100   100   120   150   150   150   200   200   200   200   200  
Orders                         120    80    50    30    20    10    0     0     0     0     0     0   
Scheduled receipts              70    0     0     0     0     0     0     0     0     0     0     0   
PAB                      160   110    10    10    10    10    10    10    10    10    10    10    10  
ATP                             30    -     50    90   130   140   150   200   200   200   200   200  
Net requirements                0     0    100   120   150   150   150   200   200   200   200   200  
Planned receipts                0     0    100   120   150   150   150   200   200   200   200   200  
Planned order release          100   120  

## Update

- MPS의 Forecast 및 Order 값 업데이트 >>> Demand 수정
- 모든 Part의 Scheduled receipt 업데이트 >>> Supply 수정

In [15]:
def update_mps(name, new_fc=None, new_od=None):
    
    global all_parts
    
    for part in all_parts:
        if part.name == name:
            if new_fc is not None:
                part.fc.update(new_fc)
            if new_od is not None:
                part.od.update(new_od)
            

def update_sr(name, new_sr=None):
    
    global all_parts
    
    for part in all_parts:
        if part.name == name:
            if new_sr is not None:
                part.sr.update(new_sr)

## Update Examples

- update_mps('MPS 이름', new_od={period:new order value}, new_fc={period:new forecast value}
- update_sr('Part 이름', new_sr={period:new scheduled receipt value}

In [16]:
# A에서 1주차의 order가 120에서 110으로 감소, 3주차의 order가 50에서 150으로 증가
# A에서 4주차의 forecaste도 120에서 150으로 증가시키기로
update_mps('a', new_od={1:110, 3:150}, new_fc={4:150})

# D에서 1주차에 150개의 scheduled receipts 발생
update_sr('d', new_sr={1:150})


set_parts()
solver()
add_solver()

record_print()

MPS                           
Product A.                0     1     2     3     4     5     6     7     8     9     10    11    12  
------------------------------------------------------------------------------------------------------
Forecasts                      100   100   100   150   150   150   150   200   200   200   200   200  
Orders                         110    80   150    30    20    10    0     0     0     0     0     0   
Scheduled receipts              70    0     0     0     0     0     0     0     0     0     0     0   
PAB                      160   120    20    10    10    10    10    10    10    10    10    10    10  
ATP                             40    -     0    120   130   140   150   200   200   200   200   200  
Net requirements                0     0    140   150   150   150   150   200   200   200   200   200  
Planned receipts                0     0    140   150   150   150   150   200   200   200   200   200  
Planned order release          140   150  

In [17]:
# A에서 2주차의 order가 80에서 70으로 감소
# B에서 2주차의 order가 120에서 100으로 감소, 3주차 order 80에서 100으로 증가
update_mps('a', new_od={2:80})
update_mps('b', new_od={2:100, 3:100})

set_parts()
solver()
add_solver()

record_print()

MPS                           
Product A.                0     1     2     3     4     5     6     7     8     9     10    11    12  
------------------------------------------------------------------------------------------------------
Forecasts                      100   100   100   150   150   150   150   200   200   200   200   200  
Orders                         110    80   150    30    20    10    0     0     0     0     0     0   
Scheduled receipts              70    0     0     0     0     0     0     0     0     0     0     0   
PAB                      160   120    20    10    10    10    10    10    10    10    10    10    10  
ATP                             40    -     0    120   130   140   150   200   200   200   200   200  
Net requirements                0     0    140   150   150   150   150   200   200   200   200   200  
Planned receipts                0     0    140   150   150   150   150   200   200   200   200   200  
Planned order release          140   150  